# CNN + NTK + CIFAR10 + FSL
Purpose: Fit a NTK to a CNN intended for the CIFAR10 dataset. The purpose is to establish a baseline for the CIFAR10 experiments.

This is the FSL variant, which means that the model from the accuracy variant is now trained using only very limited data.

Furthermore, the PyTorch Lightning library is used for convenience.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
from torch import nn
import lightning as L
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt

import sys
sys.path.append('convkans/kan_convolutional')
from KANLinear import *

# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setup Randomness -- https://lightning.ai/docs/pytorch/stable/common/trainer.html
L.seed_everything(68812, workers=True) # <--- Adjust the seed here for each trial run for the 5 seed values [42, 998244353, 10122024, 1000000007, 68812].

# CUDA Efficiency
if torch.cuda.is_available():
    torch.set_float32_matmul_precision('high')
    print("Configured CUDA Precision")

Seed set to 68812


Configured CUDA Precision


In [2]:
# Dataset Setup -- Inspired by Hugo's Dataset Reformatting
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = CIFAR10("./Experiments/temp/", train=True, download=True, transform=transform)
test_dataset = CIFAR10("./Experiments/temp/", train=False, download=True, transform=transform)

train_size = int(len(train_dataset) * 0.90)
val_size = int(len(train_dataset)) - train_size # Quite low validation size, but this is done to allow for greater model accuracy.
train_set, val_set = random_split(train_dataset, [train_size, val_size])

# Reformatted, due to odd issues when using NTK on it
class LCDataset(Dataset): # Lightning Compatible Dataset
    def __init__(self, dataset, num_classes, limit_per_class=-1, randomise=False):
        self.limit_per_class = limit_per_class
        self.num_classes = num_classes
        if self.limit_per_class != -1:
            sub = list(np.random.permutation(np.arange(len(dataset)))) # Generate a random permutation of the dataset.
            # Select the first x of each class
            sub2 = []
            sub_count = [0 for x in range(self.num_classes)]
            for x in sub:
                temp_x, temp_y = dataset[x]
                if sub_count[temp_y] != self.limit_per_class:
                    sub_count[temp_y] += 1
                    sub2.append(x)
            for x in sub_count:
                assert x == self.limit_per_class
            assert len(sub2) == self.num_classes * self.limit_per_class
            self.dataset = Subset(dataset, sub2)
        elif randomise == True: # Idea: Take 2 values each to form a pair to check against (FSL case)
            sub = list(np.random.permutation(np.arange(len(dataset)))) # Generate a random permutation of the dataset.
            self.dataset = Subset(dataset, sub)
        else:
            self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        y_one_hot = torch.zeros(self.num_classes)
        y_one_hot[y] = 1
        return x, y_one_hot

batch_size = 64
ntk_loader = DataLoader(LCDataset(train_dataset, num_classes=10, limit_per_class=15), batch_size=batch_size, num_workers=10)
train_loader = DataLoader(LCDataset(train_set, num_classes=10), batch_size=batch_size, shuffle=True, num_workers=10)
val_loader = DataLoader(LCDataset(val_set, num_classes=10), batch_size=batch_size, shuffle=False, num_workers=10)
test_loader = DataLoader(LCDataset(test_dataset, num_classes=10), batch_size=batch_size, num_workers=10)
similarity_loader = DataLoader(LCDataset(test_dataset, num_classes=10, randomise=True), batch_size=2, num_workers=10) # Each batch constitutes a pair

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Model Declaration
class ClassicCNN(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 16, kernel_size=2, padding=0, bias=False),
            nn.BatchNorm2d(16),
            nn.GELU(),
            nn.Conv2d(16, 64, kernel_size=3, padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.GELU(),
            nn.Conv2d(64, 96, kernel_size=3, padding=0, bias=False),
            nn.BatchNorm2d(96),
            nn.MaxPool2d(2),
            nn.GELU(),
            nn.Flatten(1),
            nn.Dropout(p=0.50),
            nn.Linear(3456, 64, bias=False),
            nn.GELU(),
            nn.Dropout(p=0.50),
            nn.Linear(64, 10, bias=False)
        )
    
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = F.cross_entropy(y_pred, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        loss = F.cross_entropy(y_pred, y)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", accuracy, prog_bar=True) # Observe model accuracy with time

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        v1 = torch.argmax(y_pred, dim=1)
        v2 = torch.argmax(y, dim=1)
        accuracy = torch.sum(torch.eq(v1, v2)) / len(y)
        self.log("accuracy", accuracy) # Accuracy for Accuracy vs. Parameter Experiment

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        return self(batch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-4)
        return optimizer

In [4]:
# Train + Test + Results
model = ClassicCNN()
trained_model = L.Trainer(max_epochs=0, deterministic=True, logger=CSVLogger(save_dir="Experiments/logs", name="CIFAR10NTKCNNFSL", version="fin"), callbacks=[EarlyStopping(monitor="val_loss", mode="min", patience=5)], log_every_n_steps=1)
trained_model.fit(model, train_loader, val_loader)
trained_model.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 286 K  | train
--------------------------------------------
286 K     Trainable params
0         Non-trainable params
286 K     Total params
1.148     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=0` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.11289999634027481    │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.11289999634027481}]

In [5]:
# Apply NTK
%run -i './Experiments/introduction_code_modified.py'

def cross_entropy_loss_batch(y_hat, y):
    return F.cross_entropy(y_hat, y, reduction='none')

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
ntk_model = GaussianFit(model=model, device=device, noise_var=0.0)
ntk_model.fit(ntk_loader, optimizer, MSELoss_batch) # MSELoss seems to work better for NTK Models

In [6]:
def check_ntk_acc(model, dataloader):
    res = 0.0
    sumlength = 0
    model.eval()
    model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        sumlength += len(x)
        res += (torch.argmax(model.forward(x), dim=1) == torch.argmax(y, dim=1)).sum()
    model.train()
    return res / sumlength

In [7]:
print(f'NTK Accuracy: {check_ntk_acc(ntk_model, test_loader)}')

NTK Accuracy: 0.10520000010728836


In [8]:
def check_fsl_similarity(model, dataloader): # Must use a batch size of 2 for this to work
    res = 0.0
    sumlength = 0
    model.eval()
    model.to(device)
    for it in iter(dataloader):
        x, y = it
        if len(x) != 2:
            print(f'Broke loop at: {sumlength} runs.')
            break
        x = x.to(device)
        y = y.to(device)
        fx = model.forward(x)
        if (torch.argmax(fx[0]) != torch.argmax(fx[1]) and torch.argmax(y[0]) != torch.argmax(y[1])) or (torch.argmax(fx[0]) == torch.argmax(fx[1]) and torch.argmax(y[0]) == torch.argmax(y[1])):
            res += 1
        sumlength += 1
    return res, sumlength

In [9]:
resntk, sumlengthntk = check_fsl_similarity(ntk_model, similarity_loader)
print(f'NTK FSL Score: {resntk / sumlengthntk} on {sumlengthntk} random samples')

NTK FSL Score: 0.396 on 5000 random samples


In [10]:
# R: True Value, C: Predicted Value
def make_predict_matrix(model, dataloader):
    res = np.zeros(shape=(10, 10), dtype=int)
    model = model.to(device)
    for it in iter(dataloader):
        x, y = it
        x = x.to(device)
        y = y.to(device)
        x_arg = torch.argmax(model.forward(x), dim=1)
        y_arg = torch.argmax(y, dim=1)
        for i in range(len(x_arg)):
            res[y_arg[i], x_arg[i]] += 1
    return res

In [11]:
print(make_predict_matrix(ntk_model, test_loader))

[[  0  13   0   0 614   0   0 373   0   0]
 [  0  16   0   0 866   0   0 118   0   0]
 [  0  14   0   0 696   0   0 290   0   0]
 [  0  32   0   0 711   0   0 257   0   0]
 [  0  15   0   0 804   0   0 181   0   0]
 [  0  28   0   0 647   0   0 325   0   0]
 [  0  13   0   0 834   0   0 153   0   0]
 [  0  25   0   0 743   0   0 232   0   0]
 [  0  18   0   0 790   0   0 192   0   0]
 [  0  19   0   0 887   0   0  94   0   0]]
